In [1]:
from collections import Counter, OrderedDict
import numpy as np

In [8]:
with open("coursera_sessions_train.txt") as f:
    raw_train = f.readlines()

In [10]:
for i in range(len(raw_train)):
    raw_train[i]=raw_train[i].strip()

In [104]:
raw_train_views = [s.split(';')[0] for s in raw_train]
raw_train_buys = [s.split(';')[1].strip() for s in raw_train]

In [105]:
train_views = [line.split(',') for line in raw_train_views]

In [106]:
train_buys = [line.split(',') for line in raw_train_buys]

In [113]:
with open("coursera_sessions_test.txt") as f:
    raw_test = f.readlines()
for i in range(len(raw_test)):
    raw_test[i]=raw_test[i].strip()
raw_test_views = [s.split(';')[0] for s in raw_test]
raw_test_buys = [s.split(';')[1].strip() for s in raw_test]
test_views = [line.split(',') for line in raw_test_views]
test_buys = [line.split(',') for line in raw_test_buys]

In [30]:
# flatten all lists:
train_views_flatten = []
for session in train_views:
    for val in session:
        train_views_flatten.append(val)
        
train_buys_flatten = []
for session in train_buys:
    for val in session:
        if val!='':
            train_buys_flatten.append(val)

In [31]:
print train_buys_flatten[:10]

['67', '60', '63', '86', '199', '303', '352', '519', '603', '604']


In [42]:
class OrderedCounter(Counter, OrderedDict):
    'Counter that remembers the order elements are first encountered'

    def __repr__(self):
        return '%s(%r)' % (self.__class__.__name__, OrderedDict(self))

    def __reduce__(self):
        return self.__class__, (OrderedDict(self),)

In [43]:
views_cnt = OrderedCounter(train_views_flatten)
buys_cnt=OrderedCounter(train_buys_flatten)

In [124]:
buys_cnt['170']

5

In [54]:
# getting index of the key
views_cnt.keys().index('9')

6

In [125]:
# this param is needed for future sorting trick
MAX = len(buys_cnt.keys())
print MAX

4479


In [84]:
def recommend_by_popularity(items):
    #remove duplicates
    items = list(set(items))
    scores = []
    orders = []
    for item in items: 
        if not views_cnt.has_key(item):
            scores.append(0)
            orders.append(0)    
        else:
            scores.append(views_cnt[item])
            orders.append(abs(views_cnt.keys().index(item)-MAX))
    #print items
    #print scores
    #print orders*/
    res = [i for (s,o,i) in sorted(zip(scores,orders,items), key=lambda pair: pair[:2], reverse=True)]
    #print res
    return res

In [85]:
recommend_by_popularity(train_views[1])

['12', '9', '10', '11']

In [126]:
def recommend_by_buying(items):
    #remove duplicates
    items = list(set(items))
    scores = []
    orders = []
    for item in items: 
        if not buys_cnt.has_key(item):
            scores.append(0)
        else:
            scores.append(buys_cnt[item])
        if not buys_cnt.has_key(item):
            orders.append(0)
        else:
            orders.append(abs(buys_cnt.keys().index(item)-MAX))
    #print items
    #print scores
    #print orders
    res = [i for (s,o,i) in sorted(zip(scores,orders,items), key=lambda pair: pair[:2], reverse=True)]
    #print res
    return res

In [127]:
recommend_by_buying(train_views[0])

['5', '1', '0', '3', '2', '4']

In [100]:
def precision_and_recall(recommendations, buys):
    top1=recommendations[0]
    top5=recommendations[:5]
    buys_cnt = len(buys)
    precision1=0.
    precision5=0.
    
    if top1 in buys:
        precision1=1.
    top5buys=0.
    for item in top5:
        if item in buys:
            top5buys+=1
    precision5=top5buys/5
    
    recall1 = precision1/buys_cnt
    recall5 = top5buys/buys_cnt
    return precision1, precision5, recall1, recall5

In [101]:
print train_views[7],train_buys[7]
precision_and_recall(recommend_by_popularity(train_views[7]),train_buys[7])

['59', '60', '61', '62', '60', '63', '64', '65', '66', '61', '67', '68', '67'] ['67', '60', '63']


(1.0, 0.4, 0.3333333333333333, 0.6666666666666666)

In [110]:
all_p1=[]
all_p5=[]
all_r1=[]
all_r5=[]
for i in range(len(train_views)):
    views = train_views[i]
    buys = train_buys[i]
    if len(buys)==1 and buys[0]=='':
        continue
    pr1, pr5, rec1, rec5 = precision_and_recall(recommend_by_popularity(views), buys)
    all_p1.append(pr1)
    all_p5.append(pr5)
    all_r1.append(rec1)
    all_r5.append(rec5)

TypeError: not all arguments converted during string formatting

In [111]:
s = "%.2f %.2f %.2f %.2f" % (np.mean(all_r1), np.mean(all_p1), np.mean(all_r5), np.mean(all_p5))
print s
with open("w4_popularity_train.txt",'w') as f:
    f.write(s)

0.44 0.51 0.82 0.21


In [128]:
all_p1=[]
all_p5=[]
all_r1=[]
all_r5=[]
for i in range(len(train_views)):
    views = train_views[i]
    buys = train_buys[i]
    if len(buys)==1 and buys[0]=='':
        continue
    pr1, pr5, rec1, rec5 = precision_and_recall(recommend_by_buying(views), buys)
    all_p1.append(pr1)
    all_p5.append(pr5)
    all_r1.append(rec1)
    all_r5.append(rec5)
    
s = "%.2f %.2f %.2f %.2f" % (np.mean(all_r1), np.mean(all_p1), np.mean(all_r5), np.mean(all_p5))
print s
with open("w4_buying_train.txt",'w') as f:
    f.write(s)

0.68 0.79 0.93 0.25


In [114]:
all_p1=[]
all_p5=[]
all_r1=[]
all_r5=[]
for i in range(len(test_views)):
    views = test_views[i]
    buys = test_buys[i]
    if len(buys)==1 and buys[0]=='':
        continue
    pr1, pr5, rec1, rec5 = precision_and_recall(recommend_by_popularity(views), buys)
    all_p1.append(pr1)
    all_p5.append(pr5)
    all_r1.append(rec1)
    all_r5.append(rec5)
    
s = "%.2f %.2f %.2f %.2f" % (np.mean(all_r1), np.mean(all_p1), np.mean(all_r5), np.mean(all_p5))
print s
with open("w4_popularity_test.txt",'w') as f:
    f.write(s)

0.41 0.48 0.80 0.20


In [129]:
all_p1=[]
all_p5=[]
all_r1=[]
all_r5=[]
for i in range(len(test_views)):
    views = test_views[i]
    buys = test_buys[i]
    if len(buys)==1 and buys[0]=='':
        continue
    pr1, pr5, rec1, rec5 = precision_and_recall(recommend_by_buying(views), buys)
    all_p1.append(pr1)
    all_p5.append(pr5)
    all_r1.append(rec1)
    all_r5.append(rec5)
    
s = "%.2f %.2f %.2f %.2f" % (np.mean(all_r1), np.mean(all_p1), np.mean(all_r5), np.mean(all_p5))
print s
with open("w4_buying_test.txt",'w') as f:
    f.write(s)

0.41 0.48 0.79 0.20
